In [93]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from random import randint
import random
from time import sleep
from pandas import json_normalize
import pandas as pd
import pickle
import ast
from sklearn.preprocessing import StandardScaler

#### Import 100 hot songs

In [2]:
hot100songs = pd.read_csv('100songs.csv')
hot100songs

,title,artists
0,Flowers,Miley Cyrus
1,Last Night,Morgan Wallen
2,Fast Car,Luke Combs
3,Like Crazy (Deep House Remix),Jimin
4,Like Crazy (UK Garage Remix),Jimin
...,...,...
95,"Going, Going, Gone",Luke Combs
96,Calm Down,Rema
97,Heart Attack (Rock Version),Demi Lovato
98,Tennessee Orange,Megan Moroney


#### Import clustered song collection

In [5]:
clustered_songs = pd.read_csv('clustered_songs.csv')
clustered_songs

,title,artists,uri,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,cluster
0,Take Me To Church,['Hozier'],spotify:track:7dS5EaCoMnN7DzlpT6aRn2,0.566,0.664,4,-5.303,0,0.0464,0.634000,...,0.1160,0.437,128.945,audio_features,7dS5EaCoMnN7DzlpT6aRn2,https://api.spotify.com/v1/tracks/7dS5EaCoMnN7...,https://api.spotify.com/v1/audio-analysis/7dS5...,241688,4,0
1,Cooler Than Me - Single Mix,"['Mike Posner', 'Gigamesh']",spotify:track:2V4bv1fNWfTcyRJKmej6Sj,0.768,0.820,7,-4.630,0,0.0474,0.179000,...,0.6890,0.625,129.965,audio_features,2V4bv1fNWfTcyRJKmej6Sj,https://api.spotify.com/v1/tracks/2V4bv1fNWfTc...,https://api.spotify.com/v1/audio-analysis/2V4b...,213293,4,0
2,See You Again (feat. Kali Uchis),"['Tyler, The Creator', 'Kali Uchis']",spotify:track:7KA4W4McWYRpgf0fWsJZWB,0.558,0.559,6,-9.222,1,0.0959,0.371000,...,0.1090,0.620,78.558,audio_features,7KA4W4McWYRpgf0fWsJZWB,https://api.spotify.com/v1/tracks/7KA4W4McWYRp...,https://api.spotify.com/v1/audio-analysis/7KA4...,180387,4,1
3,Pompeii,['Bastille'],spotify:track:3gbBpTdY8lnQwqxNCcf795,0.679,0.715,9,-6.383,1,0.0407,0.075500,...,0.2710,0.571,127.435,audio_features,3gbBpTdY8lnQwqxNCcf795,https://api.spotify.com/v1/tracks/3gbBpTdY8lnQ...,https://api.spotify.com/v1/audio-analysis/3gbB...,214148,4,2
4,Hips Don't Lie (feat. Wyclef Jean),"['Shakira', 'Wyclef Jean']",spotify:track:3ZFTkvIE7kyPt6Nu3PEa7V,0.778,0.824,10,-5.892,0,0.0707,0.284000,...,0.4050,0.758,100.024,audio_features,3ZFTkvIE7kyPt6Nu3PEa7V,https://api.spotify.com/v1/tracks/3ZFTkvIE7kyP...,https://api.spotify.com/v1/audio-analysis/3ZFT...,218093,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7982,Hot N Cold,['Katy Perry'],spotify:track:1ulXmqjktxSdrdT1bzc69H,0.706,0.841,7,-3.956,1,0.0418,0.000079,...,0.0688,0.861,132.032,audio_features,1ulXmqjktxSdrdT1bzc69H,https://api.spotify.com/v1/tracks/1ulXmqjktxSd...,https://api.spotify.com/v1/audio-analysis/1ulX...,220227,4,2
7983,What Lovers Do,"['Maroon 5', 'SZA']",spotify:track:0ExtPzAWi4arQz6kaabm02,0.799,0.597,5,-5.131,0,0.0611,0.078800,...,0.1000,0.419,110.001,audio_features,0ExtPzAWi4arQz6kaabm02,https://api.spotify.com/v1/tracks/0ExtPzAWi4ar...,https://api.spotify.com/v1/audio-analysis/0Ext...,199849,4,0
7984,You Need To Calm Down - Clean Bandit Remix,"['Taylor Swift', 'Clean Bandit']",spotify:track:7Bjf96uNizpvXLvNyOJpst,0.726,0.880,2,-4.137,1,0.0581,0.072000,...,0.0906,0.739,95.037,audio_features,7Bjf96uNizpvXLvNyOJpst,https://api.spotify.com/v1/tracks/7Bjf96uNizpv...,https://api.spotify.com/v1/audio-analysis/7Bjf...,152773,4,2
7985,Barracuda,['Heart'],spotify:track:4KfSdst7rW39C0sfhArdrz,0.548,0.691,4,-13.553,0,0.0369,0.015000,...,0.1440,0.667,137.145,audio_features,4KfSdst7rW39C0sfhArdrz,https://api.spotify.com/v1/tracks/4KfSdst7rW39...,https://api.spotify.com/v1/audio-analysis/4KfS...,261933,4,0


#### Function from part 2

In [13]:
def SongRecommender1():
    song_input = input("Enter your favorite song: ")
    if song_input in hot100songs['title'].unique():
        random_song = hot100songs['title'][random.randint(0, len(hot100songs)-1)]
        random_artist = hot100songs.loc[hot100songs.title == random_song, 'artists'].values[0]
        return 'Look at this new hot song: '+random_song+' from '+random_artist
    else:
        return 'Sorry, cannot give any recommendation. Try another one'

In [14]:
#Test it
SongRecommender1()

Enter your favorite song: Flowers


'Look at this new hot song: Love You Anyway from Luke Combs'

#### Load kmeans model

In [15]:
kmeans = pickle.load(open('kmeans_model.sav', 'rb'))
print(kmeans)

KMeans(n_clusters=5, random_state=1234)


#### Load Scaler

In [16]:
scaler = pickle.load(open('standard_scaler.sav', 'rb'))
print(scaler)

StandardScaler()


#### Connection to Spotify API

In [17]:
secrets_file = open("C:/Users/lisab/OneDrive/Desktop/DA_labs/Week_6/Spotify_API.txt")

In [18]:
string = secrets_file.read()

In [19]:
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()

In [20]:
#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['Client ID'],
                                                           client_secret=secrets_dict['Client Secret']))

#### Develop else part of SongRecommender Function

In [94]:
def getArtists(artist_list):
    #convert string containing list to list
    artist_list = ast.literal_eval(artist_list)
    if len(artist_list) > 1:
        artists = artist_list.join(', ')
    else:
        artists = artist_list[0]
    return artists

In [95]:
def checkSongAndRecommend():
    song_input = input("Enter your favorite song: ")
    if song_input in hot100songs['title'].unique():
        random_song = hot100songs['title'][random.randint(0, len(hot100songs)-1)]
        random_artist = hot100songs.loc[hot100songs.title == random_song, 'artists'].values[0]
        return 'Look at this new hot song: '+random_song+' from '+random_artist
    else:
        song_spotify = sp.search(q=song_input, limit=1)
        song_uri = song_spotify['tracks']['items'][0]['uri']
        song_audio_features = sp.audio_features(song_uri)
        audio_feat_df = pd.DataFrame(song_audio_features)
        audio_feat_df_corr = audio_feat_df[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 
                          'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']].copy()
        audio_feat_df_corr_prep = scaler.transform(audio_feat_df_corr)
        # Predicting / assigning the clusters:
        clusters = kmeans.predict(audio_feat_df_corr_prep)
        #take clustered_songs dataframe and search for a song with the same number as clusters_test
        clustered_songs_input_cluster = clustered_songs[clustered_songs['cluster'] == clusters[0]]
        random_song_output = clustered_songs_input_cluster['title'][random.randint(0, len(clustered_songs_input_cluster)-1)]
        random_artist_output = clustered_songs_input_cluster.loc[clustered_songs_input_cluster.title == random_song_output, 
                                'artists'].values[0]
        random_artist_output = getArtists(random_artist_output)
        return 'Found a similar song: '+random_song_output+' from '+random_artist_output

In [34]:
song_spotify = sp.search(q='Flowers', limit=1)
song_uri = song_spotify['tracks']['items'][0]['uri']
song_audio_features = sp.audio_features(song_uri)

In [35]:
song_audio_features

[{'danceability': 0.707,
  'energy': 0.681,
  'key': 0,
  'loudness': -4.325,
  'mode': 1,
  'speechiness': 0.0668,
  'acousticness': 0.0632,
  'instrumentalness': 5.15e-06,
  'liveness': 0.0322,
  'valence': 0.646,
  'tempo': 117.999,
  'type': 'audio_features',
  'id': '0yLdNVWF3Srea0uzk55zFn',
  'uri': 'spotify:track:0yLdNVWF3Srea0uzk55zFn',
  'track_href': 'https://api.spotify.com/v1/tracks/0yLdNVWF3Srea0uzk55zFn',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0yLdNVWF3Srea0uzk55zFn',
  'duration_ms': 200455,
  'time_signature': 4}]

In [38]:
audio_feat_test = pd.DataFrame(song_audio_features)
audio_feat_test

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.707,0.681,0,-4.325,1,0.0668,0.0632,0.000005,0.0322,0.646,117.999,audio_features,0yLdNVWF3Srea0uzk55zFn,spotify:track:0yLdNVWF3Srea0uzk55zFn,https://api.spotify.com/v1/tracks/0yLdNVWF3Sre...,https://api.spotify.com/v1/audio-analysis/0yLd...,200455,4


In [39]:
audio_feat_df_test = audio_feat_test[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 
                          'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']].copy()
audio_feat_df_test

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,0.707,0.681,0,-4.325,1,0.0668,0.0632,0.000005,0.0322,0.646,117.999,200455


In [40]:
from sklearn.preprocessing import StandardScaler
audio_feat_df_prep_test = scaler.transform(audio_feat_df_test)

# Now, all features will have the same weight.
pd.DataFrame(audio_feat_df_prep_test,columns=audio_feat_df_test.columns).head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,0.419048,-0.0398,-1.451287,0.709486,0.849752,-0.322207,-0.504434,-0.233535,-1.071328,0.532931,-0.142623,-0.312557


In [41]:
# Predicting / assigning the clusters:
clusters_test = kmeans.predict(audio_feat_df_prep_test)

In [42]:
clusters_test

array([2])

In [43]:
clusters_test[0]

2

In [44]:
#take clustered_songs dataframe and search for a song with the same number as clusters_test
clustered_songs_input_cluster = clustered_songs[clustered_songs['cluster'] == clusters_test[0]]
clustered_songs_input_cluster

,title,artists,uri,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,cluster
3,Pompeii,['Bastille'],spotify:track:3gbBpTdY8lnQwqxNCcf795,0.679,0.715,9,-6.383,1,0.0407,0.075500,...,0.2710,0.571,127.435,audio_features,3gbBpTdY8lnQwqxNCcf795,https://api.spotify.com/v1/tracks/3gbBpTdY8lnQ...,https://api.spotify.com/v1/audio-analysis/3gbB...,214148,4,2
5,Hall of Fame (feat. will.i.am),"['The Script', 'will.i.am']",spotify:track:7wMq5n8mYSKlQIGECKUgTX,0.421,0.873,10,-4.343,1,0.0564,0.065400,...,0.1230,0.629,84.786,audio_features,7wMq5n8mYSKlQIGECKUgTX,https://api.spotify.com/v1/tracks/7wMq5n8mYSKl...,https://api.spotify.com/v1/audio-analysis/7wMq...,202533,4,2
6,Back Where I Belong (feat. Avicii),"['Otto Knows', 'Avicii']",spotify:track:78W8wiUIlQ2SnWY9TVowKZ,0.497,0.865,0,-3.190,1,0.0644,0.030600,...,0.4430,0.595,125.960,audio_features,78W8wiUIlQ2SnWY9TVowKZ,https://api.spotify.com/v1/tracks/78W8wiUIlQ2S...,https://api.spotify.com/v1/audio-analysis/78W8...,174203,4,2
8,Dynamite,['Taio Cruz'],spotify:track:2CEgGE6aESpnmtfiZwYlbV,0.751,0.783,4,-3.724,1,0.0859,0.003790,...,0.0360,0.816,119.975,audio_features,2CEgGE6aESpnmtfiZwYlbV,https://api.spotify.com/v1/tracks/2CEgGE6aESpn...,https://api.spotify.com/v1/audio-analysis/2CEg...,202613,4,2
10,Danza Kuduro (feat. Don Omar),"['Lucenzo', 'Don Omar']",spotify:track:1kAZhbcsXqfUjnVeqPywn2,0.706,0.890,0,-6.575,1,0.0847,0.085500,...,0.0465,0.896,130.051,audio_features,1kAZhbcsXqfUjnVeqPywn2,https://api.spotify.com/v1/tracks/1kAZhbcsXqfU...,https://api.spotify.com/v1/audio-analysis/1kAZ...,202347,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7975,ME! (feat. Brendon Urie of Panic! At The Disco),"['Taylor Swift', 'Brendon Urie', 'Panic! At Th...",spotify:track:2Rk4JlNc2TPmZe2af99d45,0.610,0.830,0,-4.105,1,0.0571,0.033000,...,0.1180,0.728,182.162,audio_features,2Rk4JlNc2TPmZe2af99d45,https://api.spotify.com/v1/tracks/2Rk4JlNc2TPm...,https://api.spotify.com/v1/audio-analysis/2Rk4...,193000,4,2
7977,Stronger (What Doesn't Kill You),['Kelly Clarkson'],spotify:track:1nInOsHbtotAmEOQhtvnzP,0.562,0.939,0,-4.282,1,0.0475,0.046000,...,0.1120,0.684,116.044,audio_features,1nInOsHbtotAmEOQhtvnzP,https://api.spotify.com/v1/tracks/1nInOsHbtotA...,https://api.spotify.com/v1/audio-analysis/1nIn...,221947,4,2
7979,19-2000 - Soulchild Remix,['Gorillaz'],spotify:track:1PpfeNl5uYRBhfFcig2Uz6,0.673,0.897,0,-3.986,1,0.0512,0.194000,...,0.2990,0.892,102.045,audio_features,1PpfeNl5uYRBhfFcig2Uz6,https://api.spotify.com/v1/tracks/1PpfeNl5uYRB...,https://api.spotify.com/v1/audio-analysis/1Ppf...,209387,4,2
7982,Hot N Cold,['Katy Perry'],spotify:track:1ulXmqjktxSdrdT1bzc69H,0.706,0.841,7,-3.956,1,0.0418,0.000079,...,0.0688,0.861,132.032,audio_features,1ulXmqjktxSdrdT1bzc69H,https://api.spotify.com/v1/tracks/1ulXmqjktxSd...,https://api.spotify.com/v1/audio-analysis/1ulX...,220227,4,2


In [57]:
random_song_test = clustered_songs_input_cluster['title'][random.randint(0, len(clustered_songs_input_cluster)-1)]
random_artist_test = clustered_songs_input_cluster.loc[clustered_songs_input_cluster.title == random_song_test, 'artists'].values[0]

In [58]:
random_song_test

'TTYLXOX'

In [78]:
random_artist_test

"['Bella Thorne']"

In [79]:
type(random_artist_test)

str

In [91]:
def getArtists(artist_list):
    #convert string containing list to list
    artist_list = ast.literal_eval(artist_list)
    if len(artist_list) > 1:
        artists = artist_list.join(', ')
    else:
        artists = artist_list[0]
    return artists

In [92]:
getArtists(random_artist_test)

'Bella Thorne'

#### Test the complete function

In [98]:
checkSongAndRecommend()

Enter your favorite song: Incomplete


'Found a similar song: Macarena from Los Del Rio'

In [99]:
checkSongAndRecommend()

Enter your favorite song: Lonely


'Found a similar song: How Do You Do ? from Crazy Frog'

In [100]:
checkSongAndRecommend()

Enter your favorite song: Fast Car


'Look at this new hot song: Escapism. from RAYE & 070 Shake'